# Analyzing CIA Factbook Data Using SQL

## Open the db
Open the db file and look at basic information about which tables are available 

In [18]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [19]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Explore the data

Look at some sample data

In [20]:
%%sql
SELECT * from facts
LIMIT 5

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


See what the min and max populations are

In [21]:
%%sql
select MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


Check which country has a population of 0 and no population growth

In [22]:
%%sql
SELECT name FROM facts
WHERE population = (SELECT MIN(population) FROM facts)

 * sqlite:///factbook.db
Done.


name
Antarctica


### Check which country has a population of 7.2 billion

In [23]:
%%sql
SELECT * FROM facts
WHERE population == (SELECT MAX(population) FROM facts)

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Determine the min, max population and population growth, excluding World so it does not skew the data

In [24]:
%%sql
select MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts
WHERE name != 'World'

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


Determine the average population and area of all countries

In [25]:
%%sql
SELECT AVG(population), AVG(area)
FROM facts
WHERE name <> 'World'

 * sqlite:///factbook.db
Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


Find countries with a higher population than average and lower area than average

In [26]:
%%sql
SELECT name
FROM facts
WHERE population > (SELECT AVG(population)
            FROM facts
            WHERE name <> 'World')
AND area < (SELECT AVG(area)
            FROM facts
            WHERE name <> 'World')

 * sqlite:///factbook.db
Done.


name
Bangladesh
Germany
Iraq
Italy
Japan
"Korea, South"
Morocco
Philippines
Poland
Spain


Find the country with the highest population
Since World is in the data set, exclude it from results

In [28]:
%%sql
SELECT name 
FROM facts
WHERE name <> 'World' 
AND population = (SELECT MAX(population) FROM facts WHERE name <> 'World')

 * sqlite:///factbook.db
Done.


name
China


Which countries have the highest ratios of water to land?

In [33]:
%%sql
SELECT 
    name, 
    1.0 * area_water / area_land AS water_to_land_ratio
FROM facts
ORDER BY water_to_land_ratio DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,water_to_land_ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623
Malawi,0.25939625850340137
Netherlands,0.22571032366565366
Uganda,0.22292237442922375
Eritrea,0.16435643564356436
Liberia,0.15623961794019933


Which countries have more water than land?

In [34]:
%%sql
SELECT
    name
FROM facts
WHERE area_water > area_land

 * sqlite:///factbook.db
Done.


name
British Indian Ocean Territory
Virgin Islands
